In [1]:
# Name the object in the Gaia-DR2-based Cantat-Gaudin et al. (2018) catalog of star clusters.
# http://vizier.hia.nrc.ca/viz-bin/VizieR-3?-source=J/A%2bA/618/A93
project_name = 'Praesepe_tails'

# This notebook will download the catalog, create a Google Sheet, populate it with the relevant data (Gaia IDs, coordinates, Gmag, and BP-RP color), download TESS FFIs, and extract CPM and SAP light curves.
# Once this is done, Run_tesscheck.ipynb will guide you through the Prot analysis.

# To use an alternative target list, comment out the "Create Target List" section and swap in your own table.
# Make sure the columns are named in the same way as required by this notebook: RA_ICRS, DE_ICRS, Source, Gmag, BP-RP.

In [2]:
import matplotlib.pyplot as plt
#%matplotlib inline
from glob import glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from astropy.coordinates import SkyCoord
import astropy.units as units
!pip install astroquery
from astroquery.vizier import Vizier
import matplotlib.gridspec as gridspec # this will need to be moved up to the imports box
from astropy.table import Table, vstack, join
from gspread.exceptions import WorksheetNotFound

# Astropy coordinate tool and units object
from astropy.coordinates import SkyCoord
import astropy.units as units

# Simple aperture photometry code
!pip install photutils

# astroquery lets us download catalogs of data from the VizieR archive. For example, the Gaia DR2 catalog is hosted there.
!pip install astroquery
from astroquery.vizier import Vizier

# This is useful for mapping numbers to letters, I think.
from string import ascii_uppercase

# We'll need to make directories and move files around.
import os

# We need to give Colab access to the data stored in our Drive.
dir_main = "/content/gdrive/Shareddrives/DouglasGroup/"
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
import sys

#### I have my own Python codes saved in folders located in /content/gdrive/My Drive/PythonCode/. I've modified this for Mark.
sys.path.append(dir_main + 'tess_check/')

#### I organize my work for different projects in folders flocated in /content/gdrive/My Drive/Projects/.  I've modified this for Mark.
dir_project = dir_main + 'tess_check/'+project_name+'/'  # Here, the variable "project" is defined in the first cell in this notebook. 


Mounted at /content/gdrive/


In [3]:
# Make a directory for the project
# If it already exists, no action is taken.
dir_exist = glob(dir_project)
if np.size(dir_exist) == 0:
  os.mkdir(dir_project)
  print('Created Dir: '+dir_project)

In [4]:
fname = os.path.join(dir_project,f"{project_name}.gsheet")
print(fname)

/content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails/Praesepe_tails.gsheet


In [5]:
#### I have my own Python codes saved in folders located in /content/gdrive/My Drive/PythonCode/. I've modified this for Mark.
sys.path.append(dir_main)
# If/when you move these programs, you will need to edit myDir.py and then update its path name in the imports in tess_tools.py and tesscheck.py
from tess_check import myDir as myDir
from tess_check import tess_cpm as tess_cpm
from tess_check import tess_tools as tess_tools
from tess_check import tesscheck as tesscheck
from tess_check import gaia_clusters as gaia_clusters

# Test to make sure myDir is working properly
dir_project_function = myDir.project_dir(project_name)
if dir_project_function != dir_project:
  print('There is a problem with the "myDir" program. It is loading the incorrect path for the project')


# while fixing things in the *.py, this is handy for reloading without having to restart the Colab session:
# from importlib import reload
# reload(tess_tools)  

Mounted at /content/gdrive
Mounted at /content/gdrive


In [6]:
# Create the Target List.

input_table_path = os.path.join(dir_project,f'Praesepe_latest_tails_tesscheck.csv') #here is where you put your target list
table_exist = glob(input_table_path)
if np.size(table_exist) == 0:
  # Load a table for our target cluster
  # This queries Cantat-Gaudin et al. (2018)
  cluster_data = gaia_clusters.cluster_catalog(project_name)[0]
  # Save as csv file
  input_catalog = cluster_data.to_pandas()
  input_catalog.to_csv(input_table_path,index=False)
else:
    input_catalog = pd.read_csv(table_exist[0])

print("Number of stars: "+str(len(input_catalog)))
input_catalog[0:3]

Number of stars: 1713


,Source,RA_ICRS,DE_ICRS,Gmag,BP-RP
0,Gaia DR2 504271599124398464,86.306986,77.055634,16.392023,2.4147
1,Gaia DR2 655146893925330816,121.092378,16.007524,16.495453,2.4415
2,Gaia DR2 1091089819680643200,121.800895,62.673843,12.784296,1.3216


In [7]:
# Build a Google Sheet for the project using the "project" project name. This sheet will be placed in the Project/"project" folder.
# When you create a sheet with this code, it is placed in the main level of your Drive. 
# This waits for it to appear then moves it to the Project folder.

import shutil
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

sheet_filename = os.path.join(dir_project,f"{project_name}.gsheet")
sheet_id_file = os.path.join(dir_project,f"{project_name}.txt")
exist = 0

if os.path.exists(sheet_id_file):
  exist = 1
  print("Project sheet exists")
  # TODO: This is a hack, need a better way to store and retrieve the key
  f = open(sheet_id_file,"r")
  doc_id = f.readline()
  f.close()
  sheet = gc.open_by_key(doc_id)
else:
  print("making sheet")
  sheet = gc.create(project_name)
  new_doc_id = sheet.id
  f = open(sheet_id_file,"w")
  f.write(new_doc_id)
  f.close()
  print("Don't forget to manually move the new sheet into the correct directory!")
#while exist == 0:
  #file_locate = glob(dir_main+sheet_name)
 # if len(file_locate)>0:
  #  print("file has been created")
   # exist = 1
    #shutil.move(dir_main+sheet_name, dir_project+sheet_name)

making sheet
Don't forget to manually move the new sheet into the correct directory!


In [8]:
sheet.id

'12qu5iS4WOM8jb-uOA4MoK0GtF1ovMXkaxzHOGaao9Io'

In [9]:
# We now need to populate the Google Sheet with the information from our input target list and create columns for the TESS analysis. 

# Write a wrapper for this
# Add test to only do this if either "forced" or it doesn't exist
# e.g., def add_info_to_sheet(update=False):
# 

try:
  targets_sheet = sheet.worksheet("Targets")
except WorksheetNotFound:
  number_of_stars = len(input_catalog)
  targets_columns = ['DR2Name','RA','Dec','Gmag','BP_RP','Prot','Prot_LS','Power_LS','TESS_Data','Notes']
  targets_sheet = sheet.add_worksheet(title='Targets',rows=number_of_stars+1,cols=len(targets_columns))

  number_of_columns = len(targets_columns)
  cell_range = ascii_uppercase[0]+'1:'+ascii_uppercase[number_of_columns-1]+'1'
  cell_list = targets_sheet.range(cell_range)
  i=0
  for cell in cell_list:
      cell.value = targets_columns[i]
      i+=1
  targets_sheet.update_cells(cell_list)

  for icol in range(5):
    cell_range = ascii_uppercase[icol]+'2:'+ascii_uppercase[icol]+str(number_of_stars+1)
    cell_list = targets_sheet.range(cell_range)
    i=0
    for cell in cell_list:
      if icol == 0: 
        cell.value = str(input_catalog['Source'][i])
      if icol == 1: 
        cell.value = input_catalog['RA_ICRS'][i] # Pay attention here to the column names of the data. You might have your table list RAs as "RA" or "RAJ2000" for example.
      if icol == 2: 
        cell.value = input_catalog['DE_ICRS'][i]
      if icol == 3: 
        if np.isfinite(input_catalog['Gmag'][i]) == False:
          cell.value=9999
        else:
          cell.value = input_catalog['Gmag'][i]
      if icol == 4: 
        if np.isfinite(input_catalog['BP-RP'][i]) == False:  # We use BP-RP color, but if you want to add other colors, you'll have to modify some things...
          cell.value = 'NaN'
        else:
          cell.value = input_catalog['BP-RP'][i]
      i+=1
    targets_sheet.update_cells(cell_list)

At this point, I like to open the sheet and clean it up:
1. Delete the empty "Sheet1" default table, or else rename it and repurpose it for whatever (e.g., pasting in a Gaia EDR3 cross-match).
2. Go to "Targets" and bold the column names, then go to View>Freeze>1row to lock it in place so you can see columns while scrolling down. 

You can do these things from python here, but I haven't coded that in.

In [10]:
# Once the Sheet is populated, you can download the TESS FFI data. This will create a number of subfolders in the project folder.
# I'm setting a brighter threshold to reduce the number of files downloaded while testing
tess_tools.download_tess(project_name,mag_cut=16.0)

Created Dir: /content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails//FFI
Created Dir: /content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails//CPM
Created Dir: /content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails//Plots
Created Dir: /content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails//Panels
Created Dir: /content/gdrive/Shareddrives/DouglasGroup/tess_check/Praesepe_tails//SAP
1713
Inflating...
3 of 1713 stars
6 of 1713 stars
7 of 1713 stars
8 of 1713 stars
Inflating...
10 of 1713 stars
Inflating...
14 of 1713 stars
15 of 1713 stars
18 of 1713 stars
19 of 1713 stars
Inflating...
21 of 1713 stars
23 of 1713 stars
24 of 1713 stars
30 of 1713 stars
Inflating...
39 of 1713 stars
Inflating...
41 of 1713 stars
44 of 1713 stars
50 of 1713 stars
51 of 1713 stars
Inflating...
52 of 1713 stars
53 of 1713 stars
55 of 1713 stars
56 of 1713 stars
59 of 1713 stars
65 of 1713 stars
67 of 1713 stars
69 of 1713 stars
Inflating...
73 of 1713 stars
7

In [11]:
# This will use the FFIs to create CPM light curves, saved as *csv files stored in the Projects/[project]/CPM/ folder.

tess_tools.make_cpm(project_name)

Star 3 of 1713
Star 10 of 1713
Star 14 of 1713
Star 21 of 1713
Star 39 of 1713
Star 41 of 1713
Star 52 of 1713
Star 73 of 1713
Star 79 of 1713
Star 83 of 1713
Star 88 of 1713
Star 143 of 1713
Star 147 of 1713
Star 152 of 1713
Star 170 of 1713
Star 198 of 1713
Star 199 of 1713
Star 226 of 1713
Star 227 of 1713
Star 259 of 1713
Star 261 of 1713
Star 270 of 1713
Star 271 of 1713
Star 281 of 1713
Star 351 of 1713
Star 368 of 1713
Star 397 of 1713
Star 441 of 1713
Star 535 of 1713
Star 554 of 1713
Star 566 of 1713
Star 649 of 1713
Star 717 of 1713
Star 1135 of 1713
Star 1141 of 1713
Star 1201 of 1713
Star 1284 of 1713
Star 1341 of 1713
Star 1369 of 1713
Star 1390 of 1713
Star 1419 of 1713
Star 1450 of 1713
Star 1455 of 1713
Star 1463 of 1713
light curve not available
light curve not available
light curve not available
Star 1480 of 1713
Star 1481 of 1713
Star 1487 of 1713
Star 1496 of 1713
Star 1501 of 1713
Star 1504 of 1713
Star 1515 of 1713
Star 1530 of 1713
Star 1540 of 1713
Star 1542 of 

In [12]:
# This will use the FFIs to create SAP light curves (simple aperture photometry), saved as *csv files stored in the Projects/[project]/SAP/ folder.
tess_tools.make_sap(project_name)

Mounted at /content/gdrive/
Number of stars: 1713
Star 3 of 1713
Star 10 of 1713
Star 14 of 1713
Star 21 of 1713
Star 39 of 1713
Star 41 of 1713
Star 52 of 1713
Star 73 of 1713
Star 79 of 1713
Star 83 of 1713
Star 88 of 1713
Star 143 of 1713
Star 147 of 1713
Star 152 of 1713
Star 170 of 1713
Star 198 of 1713
Star 199 of 1713
Star 226 of 1713
Star 227 of 1713
Star 259 of 1713
Star 261 of 1713
Star 270 of 1713
Star 271 of 1713
Star 281 of 1713
Star 351 of 1713
Star 368 of 1713
Star 397 of 1713
Star 441 of 1713
Star 535 of 1713
Star 554 of 1713
Star 566 of 1713
Star 649 of 1713
Star 717 of 1713
Star 1135 of 1713
Star 1141 of 1713
Star 1201 of 1713
Star 1284 of 1713
Star 1341 of 1713
Star 1369 of 1713
Star 1390 of 1713
Star 1419 of 1713
Star 1450 of 1713
Star 1455 of 1713
Star 1463 of 1713
light curve not available
light curve not available
light curve not available
Star 1480 of 1713
Star 1481 of 1713
Star 1487 of 1713
Star 1496 of 1713
Star 1501 of 1713
Star 1504 of 1713
Star 1515 of 1713